In [14]:
#!pip install seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

#!pip install geopy
#!pip install googletrans==3.1.0a0
from googletrans import Translator
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [48]:
train = pd.read_csv('train.csv')
submission = pd.read_csv('submission.csv')

In [19]:
#### 중복행 제거
# 범주형 변수 -> Etc, etc와 같이 정제되어있지 않은 범주형 변수를 간단히 정제 : 같은 etc로 분류되어있지만 다른 데이터라고 되어있는 것을 막기 위함
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

# 특수문자 제거, 소문자 변환, 공백 제거
def regenerate_func(str_c):
    new_str = re.sub(r"[.,~!@#$%^&*\(\)_+=\-~|\\\"\';:\?/\<\>\[\]\{\}]", " ", str_c) # 특수문자
    new_str = ''.join(new_str.split())
    new_str = new_str.strip() # 양옆 공백제거
    new_str = new_str.lower() # 소문자 변환
    
    return new_str

# 범주형 변수인 경우 -> 특수문제 제거, 소문자 변환, 공백 제거 => ex) Korea와 korea는 동일, N/A와 NA는 동일
df_copy = train.copy()
df_copy[label_columns] = df_copy[label_columns].apply(lambda x: [regenerate_func(i) if not pd.isna(i) else i for i in x])  # regenerate function 적용

# 중복 데이터 확인
duplication = df_copy.duplicated().values
duplication = [int(not i) for i in duplication] # False인 경우 1, True인 경우 0으로 변환 -> False인 경우는 중복 행 중 첫번째를 의미

# 중복 갯수 산출
idx = 0
for i in range(len(train)):
    if len(train)-1 == i:  # 마지막 행은 중복행이자 첫번째 행일 수 없음. 즉, 1 이면 그대로 1, 0이면 그대로 0으로 출력
        continue
    
    if duplication[i] == 1:  # 1이면 첫번째 행이므로 그대로 출력, idx를 i로 변환해 i번째 행이 첫번째 행임을 나타냄
        idx = i
    else:
        duplication[idx] += 1  # i번째 행이 0이면 idx로 저장해둔 즉, 중복행 중 첫번째 행이 +1 -> 갯수를 셀 수 있음

# 중복행 갯수 columns 추가
train["duplication"] = duplication
train = train[train.duplication != 0]

In [20]:
### 번역 function ------------------ 240208 추가(박지호)
def translate_func(text, attempt = 1, max_attempt = 5):
    translator = Translator()
    try:
        return translator.translate(text, dest = "en").text  # 번역
    except:          # 에러의 경우
        if attempt <= max_attempt:   # 5번 이하로 다시 시도
            return translate_func(text, attempt = attempt + 1)
        raise

## 2. customer_country

In [49]:
# 전처리 이전
train_num_na = train['customer_country'].isna().sum()
train_num_class = len(train['customer_country'].unique())
train_class = train['customer_country'].unique()

submission_num_na = submission['customer_country'].isna().sum()
submission_num_class = len(submission['customer_country'].unique())
submission_class = submission['customer_country'].unique()

In [50]:
### 국가만 추출 ------------------ 240208 추가(박지호)
# 규칙에 맞는 국가
def country_func(country_str):
    country_split = country_str.split("//")
    if len(country_str.split('//')) != 1 and country_str.split("//")[-1].strip() != '':
        country_str = country_str.split("//")[-1]
    elif len(country_str.split("/")) != 1 and country_str.split("/")[-1].strip() != "":  # 주소/주소/국가인 경우
        country_str = country_str.split("/")[-1]
    return country_str

# 규칙에 맞지 않는 국가 - 약 40분
def geo_country_func(country_str, attempt = 1, max_attempt = 5):
    time.sleep(1)  # 딜레이 -> geolocoder가 1초에 1번만 실행해야함
    
    global n      # 몇 번째 행인지
    global start_t   # 얼마나 걸리는 지
    if len(country_str.split("/")) == 4 and country_str.split("/")[-1].strip() != "":
        country_str = country_str.split("/")[-1]
    elif len(country_str.split("/")) == 3 and country_str.split("/")[1].strip() != "":
        country_str = country_str.split("/")[1]
    elif country_str.split(",")[-1].strip() != "":
        country_str = country_str.split(",")[-1]
    
    try:
        n += 1
        geo = geolocoder.geocode(country_str) # 주소 검색
        if pd.isna(geo):         # 주소가 없는 경우
            country_str = geo    # 공백
        else:
            country_str = geo[0].split(",")[-1].strip()  # 주소가 있는 경우 추출
        
        # 현재 진행상황 및 시간 출력
        if n % 100 ==0:
            print("------------------ Step {} : {} -------------------".format(n, time.time() - start_t))
            start_t = time.time()
        
        return country_str
    
    except GeocoderTimedOut:  # error인 경우
        if attempt <= max_attempt:   # 5번 이하로 재시도
            return geo_country_func(country_str, attempt=attempt+1)
        raise

# 규칙에 맞는 국가 추출
train.customer_country = [country_func(i) if not pd.isna(i) else i for i in train.customer_country]
submission.customer_country = [country_func(i) if not pd.isna(i) else i for i in submission.customer_country]

# 규칙에 맞지 않는 국가 추출
n = 0
start_t = time.time()
geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
country_train = list(set(train.customer_country)) # 국가 추출 전 raw data
conversion_train = [geo_country_func(i) if not pd.isna(i) else i for i in country_train] # 국가 추출 후 data

n = 0
start_t = time.time()
country_sub = list(set(submission.customer_country)) # 국가 추출 전 raw data
conversion_sub = [geo_country_func(i) if not pd.isna(i) else i for i in country_sub] # 국가 추출 후 data

------------------ Step 100 : 140.91909313201904 -------------------
------------------ Step 200 : 138.2425093650818 -------------------
------------------ Step 300 : 141.35652208328247 -------------------
------------------ Step 400 : 139.41183280944824 -------------------
------------------ Step 500 : 140.81400394439697 -------------------
------------------ Step 600 : 136.6665666103363 -------------------
------------------ Step 700 : 142.55149006843567 -------------------
------------------ Step 800 : 139.43302750587463 -------------------
------------------ Step 900 : 140.52976512908936 -------------------
------------------ Step 1000 : 136.80347084999084 -------------------
------------------ Step 1100 : 140.59596347808838 -------------------
------------------ Step 1200 : 142.33417010307312 -------------------
------------------ Step 1300 : 138.44559741020203 -------------------
------------------ Step 1400 : 141.7730622291565 -------------------
------------------ Step 1500 : 1

In [51]:
# 번역
conversion_trans_train = [i.split("/")[-1] if not pd.isna(i) else i for i in conversion_train]  # 규칙에 맞지 않는 국가에서 특수문자가 아직 포함되어있는 경우 국가만 추출
conversion_trans_train = [translate_func(i) if not pd.isna(i) else i for i in conversion_trans_train] # 번역
conversion_trans_sub = [i.split("/")[-1] if not pd.isna(i) else i for i in conversion_sub]  # 규칙에 맞지 않는 국가에서 특수문자가 아직 포함되어있는 경우 국가만 추출
conversion_trans_sub = [translate_func(i) if not pd.isna(i) else i for i in conversion_trans_sub] # 번역

# 카테고리명 변경
conversion_df_train = pd.DataFrame([country_train, conversion_trans_train]).T  # DataFrame 생성
conversion_df_train.columns = ["country", "conversion"]            # colunm명 생성
conversion_dict_train = conversion_df_train.set_index("country").T.to_dict()   # dict형으로 변환
conversion_re_train = [conversion_dict_train[i]["conversion"] if not pd.isna(i) else i for i in train.customer_country] # country 기준 conversion으로 변경

conversion_df_sub = pd.DataFrame([country_sub, conversion_trans_sub]).T  # DataFrame 생성
conversion_df_sub.columns = ["country", "conversion"]            # colunm명 생성
conversion_dict_sub = conversion_df_sub.set_index("country").T.to_dict()   # dict형으로 변환
conversion_re_sub = [conversion_dict_sub[i]["conversion"] if not pd.isna(i) else i for i in submission.customer_country] # country 기준 conversion으로 변경

# 국가를 찾을 수 없는 카테고리는 원래의 카테고리명으로 - 결측치 그대로
conversion_na_train = [train.customer_country.iloc[i] if pd.isna(conversion_re_train[i]) else conversion_re_train[i] for i in range(len(conversion_re_train))]
conversion_na_sub = [submission.customer_country.iloc[i] if pd.isna(conversion_re_sub[i]) else conversion_re_sub[i] for i in range(len(conversion_re_sub))]

# 국가를 찾을 수 없는 카테고리는 etc로 변경 - 결측치 0
conversion_etc_train = ["etc" if pd.isna(conversion_re_train[i]) else conversion_re_train[i] for i in range(len(conversion_re_train))]
conversion_etc_sub = ["etc" if pd.isna(conversion_re_sub[i]) else conversion_re_sub[i] for i in range(len(conversion_re_sub))]

'''
train.customer_country = conversion_etc_train ## 변경
train.to_csv("conversion_train_etc.csv", index = False)
submission.customer_country = conversion_etc_sub
submission.to_csv("conversion_sub_etc.csv", index = False)

train.customer_country = conversion_na_train ## 변경
train.to_csv("conversion_train_na.csv", index = False)
submission.customer_country = conversion_na_sub
submission.to_csv("conversion_sub_na.csv", index = False)
'''

In [75]:
# 전처리 이전과 이후 비교
process_train_num_nan_na = pd.DataFrame(conversion_na_train).isna().sum()[0]
process_train_num_nan_etc = pd.DataFrame(conversion_etc_train).isna().sum()[0]

process_train_num_class_na = len(set(conversion_na_train))
process_train_num_class_etc = len(set(conversion_etc_train))

process_submission_num_nan_na = pd.DataFrame(conversion_na_sub).isna().sum()[0]
process_submission_num_nan_etc = pd.DataFrame(conversion_etc_sub).isna().sum()[0]

process_submission_num_class_na = len(set(conversion_na_sub))
process_submission_num_class_etc = len(set(conversion_etc_sub))

print(f'train 결측값 : {train_num_na} -> {process_train_num_nan_na} -> {process_train_num_nan_etc}')
print(f'train class 수 : {train_num_class} -> {process_train_num_class_na} -> {process_train_num_class_etc}')
print(f'train class : {country_train[0:3]} -> {conversion_trans_train[0:3]}',end='\n\n')

print(f'submission 결측값 : {submission_num_na} -> {process_submission_num_nan_na} -> {process_submission_num_nan_etc}')
print(f'submission class 수 : {submission_num_class} -> {process_submission_num_class_na} -> {process_submission_num_class_etc}')
print(f'submission class : {country_sub[0:3]} -> {conversion_trans_sub[0:3]}',end='\n\n')

train 결측값 : 982 -> 982 -> 0
train class 수 : 15400 -> 323 -> 158
train class : ['4477 Progress Drive Airport Information Office /South Bend /', '124 W Oak St  /Louisville/', '1 Blachley Rd  NAB Event/Stamford/'] -> ['United States', 'United States', 'United States']

submission 결측값 : 0 -> 0 -> 0
submission class 수 : 2467 -> 86 -> 83
submission class : ['   / гр. София, Област Софи / Bulgaria', 'United States', ' UNITED STATES'] -> ['Bulgaria', 'United States', 'United States']

